In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown

In [ ]:
## parameters cell
itp_id = 300

In [ ]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE

In [ ]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

# {organization_name}

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for {organization_name}, with a map each for the morning peak, midday, and afternoon peak periods on {human_date}.
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel. Thicker segments indicate more frequent transit routes.
* State Highway Network routes are outlined in gray.
* Hover over, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

## Morning Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00')

### 20th Percentile Speeds by Segment

In [ ]:
%%capture
_m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True,
                             no_render=True
                            )

In [ ]:
%%capture
rt_day.map_gz_export()

In [ ]:
rt_day.render_spa_link()

In [ ]:
rt_day.display_spa_map()

### Variation in Speeds by Segment

* This visualization shows variation as the ratio between the 80th percentile and 20th percentile speeds in each segment
* Segments with high variation in speeds make it difficult for transit operators to set accurate schedules, and can cause inconsistent service for riders

In [ ]:
rt_day.map_variance(no_render=True)

In [ ]:
%%capture
rt_day.map_gz_export(map_type = 'variance')

In [ ]:
rt_day.render_spa_link()

In [ ]:
rt_day.display_spa_map()

## Midday

In [ ]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00')

### 20th Percentile Speeds by Segment

In [ ]:
%%capture
_m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True,
                             no_render=True
                            )

In [ ]:
%%capture
rt_day.map_gz_export()

In [ ]:
rt_day.render_spa_link()

In [ ]:
rt_day.display_spa_map()

### Variation in Speeds by Segment

* This visualization shows variation as the ratio between the 80th percentile and 20th percentile speeds in each segment
* Segments with high variation in speeds make it difficult for transit operators to set accurate schedules, and can cause inconsistent service for riders

In [ ]:
rt_day.map_variance(no_render=True)

In [ ]:
%%capture
rt_day.map_gz_export(map_type = 'variance')

In [ ]:
rt_day.render_spa_link()

In [ ]:
rt_day.display_spa_map()

## Afternoon Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

### 20th Percentile Speeds by Segment

In [ ]:
%%capture
_m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True,
                             no_render=True
                            )

In [ ]:
%%capture
rt_day.map_gz_export()

In [ ]:
rt_day.render_spa_link()

In [ ]:
rt_day.display_spa_map()

### Variation in Speeds by Segment

* This visualization shows variation as the ratio between the 80th percentile and 20th percentile speeds in each segment
* Segments with high variation in speeds make it difficult for transit operators to set accurate schedules, and can cause inconsistent service for riders

In [ ]:
rt_day.map_variance(no_render=True)

In [ ]:
%%capture
rt_day.map_gz_export(map_type = 'variance')

In [ ]:
rt_day.render_spa_link()

In [ ]:
rt_day.display_spa_map()

## Daily Speed Patterns

* This chart shows the distribution of median trip speeds throughout the day. It can help find times of day where transit riders experience the slowest overall speeds.

In [ ]:
try:
    rt_day.set_filter(start_time='06:00', end_time='22:00')
    display(Markdown(f'Median Trip Speed by Hour of Day{rt_day.filter_formatted}'))

    rt_day.chart_speeds(no_title=True);
except:
    display(Markdown(f'Chart not available for this agency'))
    pass